In [1]:
import datetime as dt
import time
import ib_insync
import pandas as pd
from ib_insync import *
util.startLoop()

In [2]:
class Data:
    
    oldprice = 0
    high = 0
    low = 0
    short = False
    long = False
    ema = 0
    price = 0
    oldprice = 0
    action = ""
    points = 0.0003
    pointsextra = 0.0005
    offset = 0.00015
    offsetextra = 0.0003
    stoploss = 0.0015
    stoporder = None
    slcounter = 0
    
    def __init__(self,symbol,currency,ib):
        self.symbol = symbol
        self.currency = currency
        self.ticker = symbol+currency
        self.ib = ib
        self.contract = Forex(symbol+currency)
        
    def executeAction(self):
        #End trading if stop loss triggered 3 times 
        if(self.slcounter == 3):
            self.action = "DONOTHING"
        if(not self.checkPositionExists() and (self.action == "CLOSESELL" or self.action == "CLOSEBUY")):
            self.action = "DONOTHING"
            self.high = 0
            self.low = 0
            self.short = False
            self.long = False
            self.slcounter += 1
        #Open logic
        if(self.action == "OPENSELL" and self.price > self.ema + self.points):
            order = MarketOrder("SELL",100000)
            self.ib.placeOrder(self.contract,order)
            self.stoporder = StopOrder("BUY",100000,self.price + self.stoploss)
            self.ib.placeOrder(self.contract,self.stoporder)
            self.action = "CLOSESELL"
            self.short = True
            self.long = False
            self.low = self.price
        elif(self.action == "OPENBUY" and self.price > self.ema - self.points):
            order = MarketOrder("BUY",100000)
            self.ib.placeOrder(self.contract,order)
            self.stoporder = StopOrder("SELL",100000,self.price - self.stoploss)
            self.ib.placeOrder(self.contract,self.stoporder)
            self.action = "CLOSEBUY"
            self.short = False
            self.long = True
            self.high = self.price
        #Close logic for small movements <5 points
        elif(self.action == "CLOSESELL" and self.low != 0 and self.low <= self.ema - self.points and self.low > self.ema - self.pointsextra):
            if(self.price > self.low + self.offset and self.price < self.ema):
                order = MarketOrder("BUY",200000)
                self.ib.placeOrder(self.contract,order)
                self.ib.cancelOrder(self.stoporder)
                self.stoporder = StopOrder("SELL",100000,self.price - self.stoploss)
                self.ib.placeOrder(self.contract,self.stoporder)
                print("CLOSED SELL AT ",self.price - self.low + self.offset)
                self.action = "CLOSEBUY"
                self.short = False
                self.long = True
                self.low = 0
                self.high = 0
        elif(self.action == "CLOSEBUY" and self.high != 0 and self.high >= self.ema + self.points and self.high < self.ema + self.pointsextra):
            if(self.price < self.high - self.offset and self.price > self.ema):
                order = MarketOrder("SELL",200000)
                self.ib.placeOrder(self.contract,order)
                self.ib.cancelOrder(self.stoporder)
                self.stoporder = StopOrder("BUY",100000,self.price + self.stoploss)
                self.ib.placeOrder(self.contract,self.stoporder)
                print("CLOSED BUY AT ",self.price - self.high - self.offset)
                self.action = "CLOSESELL"
                self.short = True
                self.long = False
                self.high = 0
                self.low = 0
        #Close logic for large movements >5 points
        elif(self.action == "CLOSESELL" and self.low != 0 and self.low <= self.ema - self.pointsextra):
            if(self.price > self.low + self.offsetextra and self.price < self.ema):
                order = MarketOrder("BUY",200000)
                self.ib.placeOrder(self.contract,order)
                self.ib.cancelOrder(self.stoporder)
                self.stoporder = StopOrder("SELL",100000,self.price - self.stoploss)
                self.ib.placeOrder(self.contract,self.stoporder)
                print("CLOSED SELL AT ",self.price - self.low + self.offsetextra," extended")
                self.action = "CLOSEBUY"
                self.short = False
                self.long = True
                self.low = 0
                self.high = 0
        elif(self.action == "CLOSEBUY" and self.high != 0 and self.high >= self.ema + self.pointsextra):
            if(self.price < self.high - self.offsetextra and self.price > self.ema):
                order = MarketOrder("SELL",200000)
                self.ib.placeOrder(self.contract,order)
                self.ib.cancelOrder(self.stoporder)
                self.stoporder = StopOrder("BUY",100000,self.price + self.stoploss)
                self.ib.placeOrder(self.contract,self.stoporder)
                print("CLOSED BUY AT ",self.price - self.high - self.offsetextra," extended")
                self.action = "CLOSESELL"
                self.short = True
                self.long = False
                self.high = 0
                self.low = 0
                
                
        
    #Updates all values and returns signal
    def getAction(self):
        self.oldprice = self.price
        contract = Contract()
        contract.symbol = self.symbol #Left
        contract.secType = "CASH"
        contract.currency = self.currency #Right
        contract.exchange = "IDEALPRO"
        bars = self.ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='2 D',
            barSizeSetting='1 min',
            whatToShow='BID',
            useRTH=True,
            formatDate=1)
        df = util.df(bars)
        self.ema = df["close"].ewm(span=10).mean().iloc[-1]
        openBid = pd.to_numeric(df['open'].iloc[-1])
        closeBid = pd.to_numeric(df['close'].iloc[-1]) 
        self.price = closeBid
        if(self.checkOpenSell(openBid,closeBid)and not self.short):
            self.action = "OPENSELL"
        elif(self.checkOpenBuy(openBid,closeBid)and not self.long):
            self.action = "OPENBUY"
        elif(self.checkCloseSell(closeBid)and self.short):
            self.updateMinMax()
            self.action = "CLOSESELL"
        elif(self.checkCloseBuy(closeBid)and self.long):
            self.updateMinMax()
            self.action = "CLOSEBUY"
        else:self.updateMinMax()
        
            
    #Checks if positions exists    
    def checkPositionExists(self):
        positionOpen = False
        positions = self.ib.positions()
        for x in positions:
            stock = x[1]
            if(stock.symbol == self.symbol and stock.currency == self.currency):
                positionOpen = True
        return positionOpen
    
    #Checks if a short postions is open
    def isShort(self): 
        short = None 
        positions = self.ib.positions()
        for x in positions:
            stock = x[1]
            price = x[2]
            if(stock.symbol == self.symbol and stock.currency == self.currency):
                if(price < 0):
                    short = True
                elif(price > 0):
                    short = False
        return short
    
    #Check for open short signal
    def checkOpenSell(self,openBid,closeBid):
        openSell = False
        if(openBid < self.ema and closeBid > self.ema and not self.checkPositionExists()):
            openSell = True
        return openSell
    
    #Check for open long signal
    def checkOpenBuy(self,openBid,closeBid):
        openBuy = False
        if(openBid > self.ema and closeBid < self.ema and not self.checkPositionExists()):
            openSell = True
        return openBuy
    
    #Check for close short signal
    def checkCloseSell(self,closeBid):
        closeSell = False
        if(self.checkPositionExists() and self.isShort and closeBid < self.ema):
            closeSell = True
        return closeSell
    
    #Check for close long signal
    def checkCloseBuy(self,closeBid):
        closeBuy = False
        if(self.checkPositionExists() and not self.isShort and closeBid > self.ema):
            closeBuy = True
        return closeBuy
    
    #Updates the min price and max prices
    def updateMinMax(self):
        if(self.action == "CLOSESELL"):
            if(self.price<self.oldprice):
                self.low = self.price
            if(self.price > self.ema):
                self.low = 0
        if(self.action == "CLOSEBUY"):
            if(self.price>self.oldprice):
                self.high = self.price
            if(self.price < self.ema):
                self.high = 0
    

In [3]:
def main():
    ib = IB()
    ib.connect('127.0.0.1', 7497, clientId=1)
    gbpusd = Data("GBP","USD",ib)
    eurusd = Data("EUR","USD",ib)
    audchf = Data("AUD","CHF",ib)
    usdchf = Data("USD","CHF",ib)
    audusd = Data("AUD","USD",ib)
    usdcad = Data("USD","CAD",ib)
    while True:
        currenttime = dt.datetime.now()
        minute = currenttime.minute
        second = currenttime.second
        if(minute%1 == 0 and second == 0):
            gbpusd.getAction()
            gbpusd.executeAction()
            ##
            eurusd.getAction()
            eurusd.executeAction()
            ##
            audchf.getAction()
            audchf.executeAction()
            ##
            usdchf.getAction()
            usdchf.executeAction()
            ##
            audusd.getAction()
            audusd.executeAction()
            ##
            usdcad.getAction()
            usdcad.executeAction()
            time.sleep(1)
        time.sleep(0.1)

In [4]:
if __name__ == "__main__":
    main()

API connection failed: ConnectionRefusedError(22, 'The remote computer refused the network connection', None, 1225, None)
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [WinError 1225] The remote computer refused the network connection